## Building a Custom Dense Layer

In this lab, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

## Imports

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np

## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [24]:
# inherit from this base class
from tensorflow.keras.layers import Layer
#from tensorflow.keras.layers import *

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype="float32"),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)
        super().build(input_shape) #this part was MISSING. WOW

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [21]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense_4/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.12773488]], dtype=float32)>, <tf.Variable 'simple_dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Let's then try using it in simple network:

In [23]:
# define the dataset...Our Hello World of DNN
#xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
#ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# use the Sequential API to build a model with our custom layer
#my_layer = SimpleDense(units=1)
#model = tf.keras.Sequential([my_layer])

# configure and train the model
#model.compile(optimizer='sgd', loss='mean_squared_error')
#model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
#print(model.predict([10.0]))

# see the updated state of the variables
#print(my_layer.variables)

In [25]:
# define the dataset...Our Hello World of DNN
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')

#https://stackoverflow.com/questions/55908188/this-model-has-not-yet-been-built-error-on-model-summary
model.build((1,1)) #case enter a single value at once

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_dense_6 (SimpleDense  (1, 1)                   2         
 )                                                               
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

1/1 [==============================] - 0s 47ms/step
[[18.981579]]
[<tf.Variable 'simple_dense_6/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973302]], dtype=float32)>, <tf.Variable 'simple_dense_6/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99172246], dtype=float32)>]


## Doing the same thing with functional API

In [27]:
from tensorflow.keras.models import Model

def build_model_with_functional():
    
    # instantiate the input Tensor
    input_layer = tf.keras.Input(shape=(1, 1))
    
    # stack the layers using the syntax: new_layer()(previous_layer). easier, but not so commom
    flatten_layer = SimpleDense(units=1)(input_layer)
    
    # declare inputs and outputs: the first layer and the last, interconnected
    func_model = Model(inputs=input_layer, outputs=flatten_layer)
    
    return func_model

In [38]:
from tensorflow.keras.models import Model

def build_model_with_functional():
    
    # instantiate the input Tensor
    input_layer = tf.keras.Input(shape=(1, 1))
    
    # stack the layers using the syntax: new_layer()(previous_layer). easier, but not so commom
    flatten_layer = SimpleDense(units=1)(input_layer)
    
    # declare inputs and outputs: the first layer and the last, interconnected
    func_model = Model(inputs=input_layer, outputs=flatten_layer)
    
    return func_model

In [39]:
model = build_model_with_functional()

In [40]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1, 1)]            0         
                                                                 
 simple_dense_10 (SimpleDens  (None, 1, 1)             2         
 e)                                                              
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [43]:
# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')

model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

ValueError: in user code:

    File "C:\Users\allan.freitas\Anaconda3\envs\allan-cv\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\allan.freitas\Anaconda3\envs\allan-cv\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\allan.freitas\Anaconda3\envs\allan-cv\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\allan.freitas\Anaconda3\envs\allan-cv\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\allan.freitas\Anaconda3\envs\allan-cv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ALLAN~1.FRE\AppData\Local\Temp\__autograph_ge